In [1]:
import json
import os

from elasticsearch import Elasticsearch
from groq import Groq
import minsearch
from IPython.display import Markdown, display
from tqdm.auto import tqdm

from utils import elastic_search, rag_elastic, rag_minsearch

***

## Rag demo with minsearch

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

In [5]:
index.fit(documents)

In [6]:
groq_client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [7]:
question = 'the course has already started, can I still enroll?'

answer = rag_minsearch(client=groq_client, index=index, question=question)

In [8]:
display(Markdown(answer))

Based on the context, the QUESTION is: the course has already started, can I still enroll?

According to the FAQ, the answer is: No, there is no mention of enrolling after the course has started. The closest related answer is from the question "Can I join the course after the start date?" which states that students can still submit homeworks even if they don't register, but there is no information on enrolling in the course after it has started.

***

### Rag demo with elasticsearch

In [9]:
elastic_search_client = Elasticsearch('http://localhost:9200')

In [10]:
elastic_search_client.info()

ObjectApiResponse({'name': 'ad15fa4654dc', 'cluster_name': 'docker-cluster', 'cluster_uuid': '05BJz8gkSReW0GAKSqVkZg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
        },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = 'course-questions'

In [ ]:
elastic_search_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for one_document in tqdm(documents):
    elastic_search_client.index(index=index_name, document=one_document)

In [13]:
answer = rag_elastic(
    client=groq_client, 
    elastic_search_client=elastic_search_client, 
    index_name=index_name,
    num_results=5,
    question=question
)

In [14]:
display(Markdown(answer))

According to the CONTEXT, the answer to your QUESTION is: Yes, you can still enroll in the course even after the start date.